# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fec30b1ab50>
├── 'a' --> tensor([[-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393]])
└── 'x' --> <FastTreeValue 0x7fec30b1a760>
    └── 'c' --> tensor([[-0.5968, -2.1363,  0.6709, -0.3247],
                        [-2.6082,  0.9464, -0.4210, -1.1703],
                        [-0.7498, -0.0342, -0.5123,  0.1853]])

In [4]:
t.a

tensor([[-0.3081, -0.8250, -0.2009],
        [-0.7405, -1.2155, -0.8393]])

In [5]:
%timeit t.a

60.8 ns ± 0.0105 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fec30b1ab50>
├── 'a' --> tensor([[ 0.3505, -1.6325,  0.7320],
│                   [-0.0654, -1.0590, -2.4611]])
└── 'x' --> <FastTreeValue 0x7fec30b1a760>
    └── 'c' --> tensor([[-0.5968, -2.1363,  0.6709, -0.3247],
                        [-2.6082,  0.9464, -0.4210, -1.1703],
                        [-0.7498, -0.0342, -0.5123,  0.1853]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.0177 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393]]),
    x: Batch(
           c: tensor([[-0.5968, -2.1363,  0.6709, -0.3247],
                      [-2.6082,  0.9464, -0.4210, -1.1703],
                      [-0.7498, -0.0342, -0.5123,  0.1853]]),
       ),
)

In [10]:
b.a

tensor([[-0.3081, -0.8250, -0.2009],
        [-0.7405, -1.2155, -0.8393]])

In [11]:
%timeit b.a

53.2 ns ± 0.0599 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4832,  0.7605, -0.0336],
               [-2.4275,  0.6052, -0.4007]]),
    x: Batch(
           c: tensor([[-0.5968, -2.1363,  0.6709, -0.3247],
                      [-2.6082,  0.9464, -0.4210, -1.1703],
                      [-0.7498, -0.0342, -0.5123,  0.1853]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.223 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

865 ns ± 9.54 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 23.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 707 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fec30b8e190>
├── 'a' --> tensor([[[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]],
│           
│                   [[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]],
│           
│                   [[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]],
│           
│                   [[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]],
│           
│                   [[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]],
│           
│                   [[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]],
│           
│                   [[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]],
│           
│                   [[-0.3081, -0.8250, -0.2009],
│                    [-0.7405, -1.2155, -0.8393]]])
└── 'x' --> <FastTreeValue 0x7fec30b8e970>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fec03fd74f0>
├── 'a' --> tensor([[-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393],
│                   [-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393],
│                   [-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393],
│                   [-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393],
│                   [-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393],
│                   [-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393],
│                   [-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393],
│                   [-0.3081, -0.8250, -0.2009],
│                   [-0.7405, -1.2155, -0.8393]])
└── 'x' --> <FastTreeValue 0x7fec087a5460>
    └── 'c' --> tensor([[-0.5968, -2.1363,  0.6709, -0.3247],
                        [-2.6082,  0.9464, -0.4210, -1.1703],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 71 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.5 µs ± 59.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]],
       
               [[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]],
       
               [[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]],
       
               [[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]],
       
               [[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]],
       
               [[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]],
       
               [[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]],
       
               [[-0.3081, -0.8250, -0.2009],
                [-0.7405, -1.2155, -0.8393]]]),
    x: Batch(
           c: tensor([[[-0.5968, -2.1363,  0.6709, -0.3247],
                       [-2.6082,  0.9464, -0.4210, -1.1703],
                       [-0.7498, -0.0342, -0.5123,  0.1853]],
         

In [26]:
%timeit Batch.stack(batches)

77.4 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393],
               [-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393],
               [-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393],
               [-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393],
               [-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393],
               [-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393],
               [-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393],
               [-0.3081, -0.8250, -0.2009],
               [-0.7405, -1.2155, -0.8393]]),
    x: Batch(
           c: tensor([[-0.5968, -2.1363,  0.6709, -0.3247],
                      [-2.6082,  0.9464, -0.4210, -1.1703],
                      [-0.7498, -0.0342, -0.5123,  0.1853],
                      [-0.5968, -2.1363,  0.6709, -0.3247],
                      [-2.6082,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 427 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
